In [1]:
owner = 'team-18'

In [2]:
import polaris as po
import datamol as dm
import numpy as np
import warnings
from sklearn.exceptions import ConvergenceWarning

warnings.filterwarnings("ignore", category=ConvergenceWarning)

/home/ubuntu/micromamba/envs/moml-ilya/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [186]:
benchmark = po.load_benchmark("polaris/pkis1-kit-wt-mut-c-1")
train, test = benchmark.get_train_test_split(featurization_fn=dm.to_fp)
#train, test = benchmark.get_train_test_split()

ys = train.y
ys = np.stack([ys[target] for target in benchmark.target_cols], axis=1)
mask = ~np.any(np.isnan(ys), axis=1)
mask.sum()

X_train = train.X[mask]
y_train = ys[mask]



2024-06-21 13:10:13.535 | INFO     | polaris._artifact:_validate_version:66 - The version of Polaris that was used to create the artifact (0.0.0) is different from the currently installed version of Polaris (dev).
2024-06-21 13:10:13.541 | INFO     | polaris._artifact:_validate_version:66 - The version of Polaris that was used to create the artifact (0.0.0) is different from the currently installed version of Polaris (dev).


In [187]:
## Balancing multi-task

from functools import partial

from imblearn.over_sampling import SMOTE
import polaris as po
import datamol as dm
import numpy as np

mapping = {
 (0.0, 0.0, 0.0): 0,
 (1.0, 0.0, 0.0): 1,
 (1.0, 0.0, 1.0): 2,
 (1.0, 1.0, 0.0): 3,
 (1.0, 1.0, 1.0): 4,
}
inv_mapping = {v: k for k, v in mapping.items()}

# load dataset
benchmark = po.load_benchmark("polaris/pkis1-kit-wt-mut-c-1")
# use ECFP fingerprint
train, test = benchmark.get_train_test_split(featurization_fn=partial(dm.to_fp, fp_type='ecfp'))

# define order of target values
target_order = ['CLASS_KIT', 'CLASS_KIT_(T6701_mutant)', 'CLASS_KIT_(V560G_mutant)']

# reshape the y values for convenience
ys = train.y
ys = np.stack([ys[target] for target in target_order], axis=1)
ys.shape

# remove the rows with NaN values
mask = ~np.any(np.isnan(ys), axis=1)
mask.sum()
X = train.X[mask]
ys = ys[mask]

ys_scalarized = [tuple(item) for item in ys]
ys_scalarized = [mapping[item] for i, item in enumerate(ys_scalarized)]

X_resampled, y_resampled = SMOTE(k_neighbors=2).fit_resample(X, ys_scalarized)
y_resampled = [inv_mapping[item] for i, item in enumerate(y_resampled)]


X_train = X_resampled
y_train = y_resampled

2024-06-21 13:10:18.234 | INFO     | polaris._artifact:_validate_version:66 - The version of Polaris that was used to create the artifact (0.0.0) is different from the currently installed version of Polaris (dev).
2024-06-21 13:10:18.238 | INFO     | polaris._artifact:_validate_version:66 - The version of Polaris that was used to create the artifact (0.0.0) is different from the currently installed version of Polaris (dev).


In [188]:
import numpy as np
import pandas as pd
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from bayes_opt import BayesianOptimization



def get_param(optimizer):
    best_params = optimizer.max['params']
    best_params['alpha'] = float(best_params['alpha'])
    best_params['length_scale'] = int(best_params['length_scale'])

    return best_params

In [191]:
def opt_target(target, param_bounds, balance = False):
    if isinstance(target, int) is True:
        target = benchmark.target_cols[target]
    

    if balance:
        benchmark = po.load_benchmark("polaris/pkis1-kit-wt-mut-c-1")
        # use ECFP fingerprint
        train, test = benchmark.get_train_test_split(featurization_fn=partial(dm.to_fp, fp_type='ecfp'))
        
        # define order of target values
        target_order = ['CLASS_KIT', 'CLASS_KIT_(T6701_mutant)', 'CLASS_KIT_(V560G_mutant)']
        
        # reshape the y values for convenience
        ys = train.y
        ys = np.stack([ys[target] for target in target_order], axis=1)
        ys.shape
        
        # remove the rows with NaN values
        mask = ~np.any(np.isnan(ys), axis=1)
        mask.sum()
        X = train.X[mask]
        
        ys = ys[mask]
    
        
        X_KIT_resampled, y_KIT_resampled = SMOTE().fit_resample(X, ys[:, 0])
        X_T670I_resampled, y_T670I_resampled = SMOTE().fit_resample(X, ys[:, 1])
        X_V560G_resampled, y_V560G_resampled = SMOTE().fit_resample(X, ys[:, 2])
    
        if target == 'CLASS_KIT_(T6701_mutant)':
            X_train = X_T670I_resampled
            y_train = y_T670I_resampled
    
        if target == 'CLASS_KIT_(V560G_mutant)':
            X_train = X_V560G_resampled
            y_train = y_V560G_resampled
    
        if target == 'CLASS_KIT':
            X_train = X_KIT_resampled
            y_train = y_KIT_resampled
        
    else:
        benchmark = po.load_benchmark("polaris/pkis1-kit-wt-mut-c-1")
        train, test = benchmark.get_train_test_split(featurization_fn=dm.to_fp)
        #train, test = benchmark.get_train_test_split()
        
        ys = train.y[target]
        mask = ~np.any(np.isnan(ys), axis=1)
        
        X_train = train.X[mask]
        y_train = ys[mask]
        


    
    
    def gp_cv(alpha, length_scale):
        gamma = float(alpha)
        length_scale = float(length_scale)
        
        kernel = C(1.0) * RBF(length_scale=length_scale)
        gp = GaussianProcessRegressor(kernel=kernel, alpha=alpha, random_state=42)
        
        # Perform cross-validation
        acc = 0
        for i in range(5):
            cval = cross_val_score(gp, X_train, y_train, scoring='neg_mean_squared_error', cv=2)
            acc += cval.mean()
        return acc/5
    

    optimizer = BayesianOptimization(
        f=gp_cv,
        pbounds=param_bounds,
        verbose=2, 
        random_state=42
    )
    optimizer.maximize(init_points=10, n_iter=5)
    best_params = get_param(optimizer)

    kernel = C(1.0) * RBF(length_scale=best_params['length_scale'])
        
    best_gp = GaussianProcessRegressor(
        kernel = kernel,
        alpha=best_params['alpha'],
        random_state=42
    )
    
    best_gp.fit(X_train, y_train)

    return best_gp, best_params


In [193]:
param_bounds = {
    'alpha': (1e-10, 1e-1),
    'length_scale': (1e0, 1e6)
}

models = {target: opt_target(target, param_bounds = param_bounds, balance = True) for target in benchmark.target_cols}

2024-06-21 13:11:21.808 | INFO     | polaris._artifact:_validate_version:66 - The version of Polaris that was used to create the artifact (0.0.0) is different from the currently installed version of Polaris (dev).
2024-06-21 13:11:21.813 | INFO     | polaris._artifact:_validate_version:66 - The version of Polaris that was used to create the artifact (0.0.0) is different from the currently installed version of Polaris (dev).


|   iter    |  target   |   alpha   | length... |
-------------------------------------------------
| 1         | -0.9264   | 0.03745   | 9.507e+05 |
| 2         | -0.9324   | 0.0732    | 5.987e+05 |
| 3         | -0.896    | 0.0156    | 1.56e+05  |
| 4         | -0.8193   | 0.005808  | 8.662e+05 |
| 5         | -0.9304   | 0.06011   | 7.081e+05 |
| 6         | -0.6385   | 0.002058  | 9.699e+05 |
| 7         | -0.9339   | 0.08324   | 2.123e+05 |
| 8         | -0.9008   | 0.01818   | 1.834e+05 |
| 9         | -0.9247   | 0.03042   | 5.248e+05 |
| 10        | -0.9275   | 0.04319   | 2.912e+05 |
| 11        | -0.9315   | 0.06734   | 9.699e+05 |
| 12        | -0.9338   | 0.08284   | 3.003e+05 |
| 13        | -0.8852   | 0.01014   | 5.217e+05 |
| 14        | -0.4963   | 0.0005644 | 4.638e+05 |
| 15        | -0.9314   | 0.06675   | 6.422e+05 |


2024-06-21 13:12:20.545 | INFO     | polaris._artifact:_validate_version:66 - The version of Polaris that was used to create the artifact (0.0.0) is different from the currently installed version of Polaris (dev).
2024-06-21 13:12:20.549 | INFO     | polaris._artifact:_validate_version:66 - The version of Polaris that was used to create the artifact (0.0.0) is different from the currently installed version of Polaris (dev).


|   iter    |  target   |   alpha   | length... |
-------------------------------------------------
| 1         | -0.9148   | 0.03745   | 9.507e+05 |
| 2         | -0.9203   | 0.0732    | 5.987e+05 |
| 3         | -0.8776   | 0.0156    | 1.56e+05  |
| 4         | -0.808    | 0.005808  | 8.662e+05 |
| 5         | -0.9185   | 0.06011   | 7.081e+05 |
| 6         | -0.468    | 0.002058  | 9.699e+05 |
| 7         | -0.9216   | 0.08324   | 2.123e+05 |
| 8         | -0.8819   | 0.01818   | 1.834e+05 |
| 9         | -0.9028   | 0.03042   | 5.248e+05 |
| 10        | -0.9159   | 0.04319   | 2.912e+05 |
| 11        | -0.9195   | 0.06734   | 9.699e+05 |
| 12        | -0.9215   | 0.08284   | 3.003e+05 |
| 13        | -0.867    | 0.01014   | 5.217e+05 |
| 14        | -0.4928   | 0.0005644 | 4.638e+05 |
| 15        | -0.9194   | 0.06675   | 6.422e+05 |


2024-06-21 13:13:18.488 | INFO     | polaris._artifact:_validate_version:66 - The version of Polaris that was used to create the artifact (0.0.0) is different from the currently installed version of Polaris (dev).
2024-06-21 13:13:18.492 | INFO     | polaris._artifact:_validate_version:66 - The version of Polaris that was used to create the artifact (0.0.0) is different from the currently installed version of Polaris (dev).


|   iter    |  target   |   alpha   | length... |
-------------------------------------------------
| 1         | -0.5485   | 0.03745   | 9.507e+05 |
| 2         | -0.6497   | 0.0732    | 5.987e+05 |
| 3         | -0.4767   | 0.0156    | 1.56e+05  |
| 4         | -0.4032   | 0.005808  | 8.662e+05 |
| 5         | -0.6329   | 0.06011   | 7.081e+05 |
| 6         | -0.4955   | 0.002058  | 9.699e+05 |
| 7         | -0.661    | 0.08324   | 2.123e+05 |
| 8         | -0.4921   | 0.01818   | 1.834e+05 |
| 9         | -0.53     | 0.03042   | 5.248e+05 |
| 10        | -0.565    | 0.04319   | 2.912e+05 |
| 11        | -0.6426   | 0.06734   | 9.699e+05 |
| 12        | -0.6606   | 0.08284   | 3.003e+05 |
| 13        | -0.4112   | 0.01014   | 5.217e+05 |
| 14        | -0.4955   | 0.0005644 | 4.638e+05 |
| 15        | -0.6418   | 0.06675   | 6.422e+05 |


In [194]:
y_prob_ind = {target: model.predict(test.X, return_std=True)[1] for target, (model, params) in models.items()}
y_pred_ind = {target: np.round(model.predict(test.X), decimals=0) for target, (model, params) in models.items()}

print(y_prob_ind)


best_params_ind = {target: params for target, (model, params) in models.items()}

results_ind = benchmark.evaluate(y_pred=y_pred_ind, y_prob=y_prob_ind)

results_ind.name = "gp_ind_cleanX"
results_ind.description = best_params_ind
results_ind.to_json('gp_ind_cleanX.json')

{'CLASS_KIT_(T6701_mutant)': array([0.70670986, 0.70670986, 0.70670986, 0.70670986, 0.70670986,
       0.70670986, 0.70670986, 0.70670986, 0.70670986, 0.70670986,
       0.70670986, 0.70670986, 0.70670986, 0.70670986, 0.70670986,
       0.70670986, 0.70670986, 0.70670986, 0.70670986, 0.70670986,
       0.70670986, 0.70670986, 0.70670986, 0.70670986, 0.70670986,
       0.70670986, 0.70670986, 0.70670986, 0.70670986, 0.70670986,
       0.70670986, 0.70670986, 0.70670986, 0.70670986, 0.70670986,
       0.70670986, 0.70670986, 0.70670986, 0.70670986, 0.70670986,
       0.70670986, 0.70670986, 0.70670986, 0.70670986, 0.70670986,
       0.70670986, 0.70670986, 0.70670986, 0.70670986, 0.70670986,
       0.70670986, 0.70670986, 0.70670986, 0.70670986, 0.70670986,
       0.70670986, 0.70670986, 0.70670986, 0.70670986, 0.70670986,
       0.70670986, 0.70670986, 0.70670986, 0.70670986, 0.70670986,
       0.70670986, 0.70670986, 0.70670986, 0.70670986, 0.70670986,
       0.70670986, 0.70670986, 0.

/home/ubuntu/micromamba/envs/moml-ilya/lib/python3.11/site-packages/pydantic/main.py:347: UserWarning: Pydantic serializer warnings:
  Expected `str` but got `dict` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


In [113]:
results_ind

{
  "name": "gp_ind",
  "description": {
    "CLASS_KIT_(T6701_mutant)": {
      "alpha": 0.000564353791300396,
      "length_scale": 13913
    },
    "CLASS_KIT_(V560G_mutant)": {
      "alpha": 0.0020584495275217957,
      "length_scale": 29097
    },
    "CLASS_KIT": {
      "alpha": 0.0135633320645141,
      "length_scale": 26673
    }
  },
  "tags": [],
  "user_attributes": {},
  "owner": null,
  "polaris_version": "dev",
  "benchmark_name": "pkis1-kit-wt-mut-c-1",
  "benchmark_owner": {
    "slug": "polaris",
    "external_id": "org_2gtoaJIVrgRqiIR8Qm5BnpFCbxu",
    "type": "organization"
  },
  "github_url": null,
  "paper_url": null,
  "contributors": null,
  "artifact_id": null,
  "benchmark_artifact_id": "polaris/pkis1-kit-wt-mut-c-1",
  "results": [
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "accuracy",
      "Score": 0.8390804598
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "accuracy",
      "Score": 0.8620689655
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "accuracy",
      "Score": 0.7126436782
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "f1",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "f1",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "f1",
      "Score": 0.4444444444
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "roc_auc",
      "Score": 0.5
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "roc_auc",
      "Score": 0.5
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "roc_auc",
      "Score": 0.4814814815
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "pr_auc",
      "Score": 0.1609195402
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "pr_auc",
      "Score": 0.1379310345
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "pr_auc",
      "Score": 0.366525254
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "mcc",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "mcc",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "mcc",
      "Score": 0.374285702
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "cohen_kappa",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "cohen_kappa",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "cohen_kappa",
      "Score": 0.3035542747
    }
  ]
}

In [181]:
def opt_multi_targets(param_bounds, balance = False):
    from sklearn.multioutput import MultiOutputRegressor
    
    if balance:
        ## Balancing multi-task

        from functools import partial
        
        from imblearn.over_sampling import SMOTE
        import polaris as po
        import datamol as dm
        import numpy as np
        
        mapping = {
         (0.0, 0.0, 0.0): 0,
         (1.0, 0.0, 0.0): 1,
         (1.0, 0.0, 1.0): 2,
         (1.0, 1.0, 0.0): 3,
         (1.0, 1.0, 1.0): 4,
        }
        inv_mapping = {v: k for k, v in mapping.items()}
        
        # load dataset
        benchmark = po.load_benchmark("polaris/pkis1-kit-wt-mut-c-1")
        # use ECFP fingerprint
        train, test = benchmark.get_train_test_split(featurization_fn=partial(dm.to_fp, fp_type='ecfp'))
        
        # define order of target values
        target_order = ['CLASS_KIT', 'CLASS_KIT_(T6701_mutant)', 'CLASS_KIT_(V560G_mutant)']
        
        # reshape the y values for convenience
        ys = train.y
        ys = np.stack([ys[target] for target in target_order], axis=1)
        ys.shape
        
        # remove the rows with NaN values
        mask = ~np.any(np.isnan(ys), axis=1)
        mask.sum()
        X = train.X[mask]
        ys = ys[mask]
        
        ys_scalarized = [tuple(item) for item in ys]
        ys_scalarized = [mapping[item] for i, item in enumerate(ys_scalarized)]
        
        X_train, y_resampled = SMOTE(k_neighbors=2).fit_resample(X, ys_scalarized)
        y_train = [inv_mapping[item] for i, item in enumerate(y_resampled)]
        
    else:
        X = train.X[mask]
        y_train = ys[mask]
    #print(y_train)

    def multi_gp_cv(alpha, length_scale):
        gamma = float(alpha)
        length_scale = float(length_scale)
        
        kernel = C(1.0) * RBF(length_scale=length_scale)
        gp = GaussianProcessRegressor(kernel=kernel, alpha=alpha, random_state=42)

        multi_gp = MultiOutputRegressor(gp, n_jobs=-1)

        acc = 0
        for i in range(5):
            # Perform cross-validation
            cval = cross_val_score(multi_gp, X_train, y_train, scoring='neg_mean_squared_error', cv=2)
            acc += cval.mean()
        return acc/5
    

    optimizer = BayesianOptimization(
        f=multi_gp_cv,
        pbounds=param_bounds,
        verbose=2, 
        random_state=42
    )
    optimizer.maximize(init_points=10, n_iter=5)
    best_params = get_param(optimizer)

    kernel = C(1.0) * RBF(length_scale=best_params['length_scale'])
        
    best_gp = GaussianProcessRegressor(
        kernel = kernel,
        alpha=best_params['alpha'],
        random_state=42
    )
    multi_gp = MultiOutputRegressor(best_gp, n_jobs=-1)
    
    multi_gp.fit(X_train, y_train)

    return multi_gp, best_params


In [182]:
param_bounds = {
    'alpha': (1e-10, 1e-1),
    'length_scale': (1e0, 1e6)
}

best_multi_gp, best_params = opt_multi_targets(param_bounds = param_bounds, balance = True)

2024-06-21 12:57:30.247 | INFO     | polaris._artifact:_validate_version:66 - The version of Polaris that was used to create the artifact (0.0.0) is different from the currently installed version of Polaris (dev).
2024-06-21 12:57:30.251 | INFO     | polaris._artifact:_validate_version:66 - The version of Polaris that was used to create the artifact (0.0.0) is different from the currently installed version of Polaris (dev).


|   iter    |  target   |   alpha   | length... |
-------------------------------------------------


/home/ubuntu/micromamba/envs/moml-ilya/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/ubuntu/micromamba/envs/moml-ilya/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/ubuntu/micromamba/envs/moml-ilya/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/hom

| 1         | -0.4259   | 0.03745   | 9.507e+05 |


/home/ubuntu/micromamba/envs/moml-ilya/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/ubuntu/micromamba/envs/moml-ilya/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/ubuntu/micromamba/envs/moml-ilya/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/hom

...

| 13        | -0.4901   | 0.004347  | 7.99e+05  |


/home/ubuntu/micromamba/envs/moml-ilya/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/ubuntu/micromamba/envs/moml-ilya/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/ubuntu/micromamba/envs/moml-ilya/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/hom

| 14        | -0.4325   | 0.03218   | 3.54e+05  |


/home/ubuntu/micromamba/envs/moml-ilya/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/ubuntu/micromamba/envs/moml-ilya/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/ubuntu/micromamba/envs/moml-ilya/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/hom

| 15        | -0.431    | 0.03327   | 4.183e+05 |


In [ ]:
Print('1')

In [183]:
y_prob_ind = {target: model.predict(test.X, return_std=True)[1] for target, (model, params) in models.items()}
y_pred_ind = {target: np.round(model.predict(test.X), decimals=0) for target, (model, params) in models.items()}

best_params_ind = {target: params for target, (model, params) in models.items()}

results_ind = benchmark.evaluate(y_pred=y_pred_ind, y_prob=y_prob_ind)

results_ind.name = "gp_multi_cleanX"
results_ind.description = best_params_ind
results_ind.to_json('gp_multi_cleanX.json')



y_pred = best_multi_gp.predict(test.X)
#y_pred.shape

y_prob = best_multi_gp.predict(test.X, return_std=True)[1]
y_prob = np.stack(y_prob, axis=1)
#y_prob.shape

y_pred_multi = {k: y_pred[:, idx] for idx, k in enumerate(benchmark.target_cols)}
y_prob_multi = {k: y_prob[:, idx, 1] for idx, k in enumerate(benchmark.target_cols)}

results_multi = benchmark.evaluate(y_pred=y_pred_multi, y_prob=y_prob_multi)


results_multi.name = "gp_multi"
results_multi.description = best_params
results_multi.to_json('gp_multi.json')

ValueError: X has 2048 features, but GaussianProcessRegressor is expecting 1784 features as input.